In [1]:
import json
from enum import Enum
from collections import namedtuple
from ipynb.fs.full.Critical_Section import *
import numpy as np

In [2]:
class lock_event:
    
    def __init__(self,n_thr):
        self.lock_clock = np.zeros(n_thr)
        self.objects = {}
    
    

### Read the events from log

In [7]:
events, critical_section_list,num_threads = gen_event_cs("temp.log")
for x in critical_section_list:
    print(x)

Thread : 1 
Locks : set() 
Read Objects : {'phi_x'} 
Write Objects : set() 
Start : 0 
End : 0 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : {'phi_y'} 
Write Objects : set() 
Start : 1 
End : 1 
Child Sections :

Thread : 1 
Locks : {'m', 'l', 'n'} 
Read Objects : {'a', 'xl', 'x'} 
Write Objects : {'y', 'b', 'yl'} 
Start : 2 
End : 17 
Child Sections :
Thread : 1 
Locks : {'l', 'n'} 
Read Objects : {'xl', 'x'} 
Write Objects : {'y', 'yl'} 
Start : 5 
End : 12 
Child Sections :
Thread : 1 
Locks : {'n'} 
Read Objects : {'xl'} 
Write Objects : {'yl'} 
Start : 8 
End : 11 
Child Sections :
Thread : 1 
Locks : {'n'} 
Read Objects : {'xl'} 
Write Objects : {'yl'} 
Start : 13 
End : 16 
Child Sections :



In [8]:
thread_cp = {}

2
